In [34]:
import os
import time
import pickle

import numpy as np
import pandas as pd
import plotly_express as px
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import sklearn.metrics as metrics
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [51]:
FEATURES = ['net_acc_std', 'net_acc_max', 'EDA_tonic_mean', 'EDA_tonic_min',  'EDA_tonic_max', 'label']
STRESS = {
    0: "amusement", 1: "baseline", 2: "stress"
}

In [63]:
def prepare_dataset(dataset_name):
    data = pd.read_csv(f'../../data/03_primary/{dataset_name}/combined_subjects.csv')
    df = data.loc[:, data.columns.intersection(FEATURES)]
    df = df[df.label != 0]
    Y_ = df.label
    X_ = df.drop(columns=['label'])
    return model_selection.train_test_split(X_, Y_, test_size=0.3, random_state=42, stratify=Y_)

In [64]:
def train(m, dataset_name):
    X, x, Y, y = prepare_dataset(dataset_name)
    start_time = time.time()
    result = m.fit(X, Y).predict(x)
    print(f"Finished in: {time.time() - start_time:.2f} seconds")
    print("Accuracy: {:.2f}%".format(metrics.accuracy_score(y, result) * 100))
    print("Precision: {0:.2f}".format(metrics.precision_score(y, result, average='macro')))
    print("Recall: {0:.2f}".format(metrics.recall_score(y, result, average='macro')))
    print("F1 Score: {0:.2f}".format(metrics.f1_score(y, result, average='macro')))
    print("R2 Score: {0:.2f}".format(metrics.r2_score(y, result)))

In [58]:
X_train, X_test, y_train, y_test = prepare_dataset('WESAD_preprocessed_int10_add10')

In [71]:
svc = LinearSVC()

param_grid={
                'C': [0.5, 1, 2],
                'loss': ['hinge', 'log_loss', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                'random_state': [42],
                'class_weight': [None, 'balanced'],
                'tol': [1e-06, 1e-05, 1e-04, 1e-03],
                'max_iter': [250, 500, 1000, 2000]
           },

model = GridSearchCV(svc, param_grid, return_train_score=True)
search = model.fit(X_train, y_train)

C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Li

In [72]:
search.best_params_

{'C': 2,
 'class_weight': 'balanced',
 'loss': 'hinge',
 'max_iter': 250,
 'random_state': 42,
 'tol': 1e-06}

In [79]:
train(LinearSVC(
    C=2,
    class_weight='balanced',
    loss='hinge',
    random_state=42,
    tol=1e-06,
    max_iter=500
), 'WESAD_preprocessed_int10_add10')

Finished in: 0.02 seconds
Accuracy: 87.78%
Precision: 0.86
Recall: 0.88
F1 Score: 0.87
R2 Score: 0.47


C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [80]:
train(LinearSVC(
    C=2,
    class_weight='balanced',
    dual=True,
    random_state=42,
    tol=1e-05,
    max_iter=1000
), 'WESAD_preprocessed_int10_add15')

Finished in: 0.10 seconds
Accuracy: 86.98%
Precision: 0.86
Recall: 0.86
F1 Score: 0.86
R2 Score: 0.43


C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [81]:
train(LinearSVC(
    C=2,
    class_weight='balanced',
    dual=True,
    random_state=42,
    tol=1e-05,
    max_iter=1000
), 'WESAD_preprocessed_int15_add15')

Finished in: 0.07 seconds
Accuracy: 88.26%
Precision: 0.87
Recall: 0.88
F1 Score: 0.87
R2 Score: 0.49


C:\Users\ratus\anaconda3\envs\ex\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
